# Module 5: Programmer un réseau de neurones convolutifs avec Keras pour le diagnistic dermatologique

Licence Professionelle "Métier du Décisionnel et de la Statistique" (2023-2024)



Dans cet exercice "grandeur nature", vous aller développer et entrainer un modèle de classification pour identifier des mélanomes à partir de photographies.

### Téléchargement des données

In [ ]:
!wget -O skin_binary.zip https://amubox.univ-amu.fr/s/EGKcZpqEtaaKXLQ/download -nv
!unzip -qq skin_binary.zip

2024-02-22 09:25:18 URL:https://amubox.univ-amu.fr/s/EGKcZpqEtaaKXLQ/download [65506845/65506845] -> "skin_binary.zip" [1]
replace images/melanoma/1228.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Création des ImageDataGenerator

Créer les générateur à partir de Keras ImageDataGenerator pour lire les images et effectuer un train/validation split de 80%/20%. Utilisez une taille de batch de 32. Faites en sortes que les images soit de taille 224x224 et en couleur.
Comme nous allons travailler ensuite avec un modèle pré-entrainé __ResNet50__, chargez et utilisez la fonction de preprocessing de cette architecture dans votre générator. Aucune augmentation de donnée n'est demandé pour ces générateurs.

In [ ]:
# Créez votre objet ImageDataGenerator dans cette cellule
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator(
  validation_split =0.2,
  preprocessing_function = preprocess_input

)

In [ ]:
# Créez votre générateur d'entrainement (train_gen) et votre générateur de validation (val_gen)

train_gen = datagen.flow_from_directory(
    "images",
    batch_size=32,
    target_size=(224,224),
    color_mode = "rgb",
    subset="training"
)


val_gen = datagen.flow_from_directory(
    "images",
    batch_size=32,
    target_size=(224,224),
    color_mode = "rgb",
    subset="validation"
)


Found 1782 images belonging to 2 classes.
Found 444 images belonging to 2 classes.


Notez le nombre d'images du jeu d'entrainement et du jeu de validation et complétez la cellule suivante avec les bonnes valeurs

In [ ]:
nb_image_training   = 1782 # Remplacez 0 par la valeur que vous avez
nb_image_validation = 444 # Remplacez 0 par la valeur que vous avez


### Création du réseau de neurone

Utilisez la librairie Keras applications pour créer un réseau de neurones basés sur l'architecture __ResNet50__, pré-entrainé sur ImageNet, et adapté aux caractéristiques de votre jeu de donnée
 (taille des images en entrée, couleur ?, combien de classes en sortie ?)

In [ ]:
# Créer la partie convolutive de votre modèle ResNet50 pré-entrainé sur imagenet dans cette cellule
from tensorflow.keras.applications import ResNet50
model0 = ResNet50(input_shape=(224,224,3), include_top=False, pooling="avg", weights="imagenet")

94765736/94765736 [==============================] - 12s 0us/step


In [ ]:
model0.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
# Ajoutez la partie classification à la couche convolutive
# en utilisant une couche cachée de taille 1024 munie de la fonction d'activation relu
# Appellez votre modèle: model

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inp = Input((224,224,3))
x = model0 (inp)                                    # model est la partie convolutive
x = Dense(1024,activation="relu") (x)               # couche Dense munie de la fonction d'activation relu
output  = Dense(2,activation="softmax") (x)        # couche Dense de sortie contenant 2 neurones pour un problème de classification avec 2 classes


model = Model (inputs = inp, outputs= output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 25687938 (97.99 MB)
Trainable params: 25634818 (97.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


### Compilation et entrainement du modèle

La compilation du modèle permet de définir:
- L'optimizer
- La fonction de loss adaptée à la problématique (ici classification)
- La métrique

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy

model.compile(Adam(learning_rate=1e-3),categorical_crossentropy,categorical_accuracy)


NameError: name 'model' is not defined

On peut désormais entrainer notre modèle sur 10 epochs par exemple

In [ ]:
H = model.fit(
    train_gen,
    validation_data=(val_gen),
    steps_per_epoch=nb_image_training//32,
    validation_steps=nb_image_validation//32,
    epochs=10
)

NameError: name 'model' is not defined

### Monitoring du modèle

Affichage de la courbe d'apprentissage

In [ ]:
import matplotlib.pyplot as plt

def learning_curves(H):
  # Plot training & validation loss values
  f = plt.figure(figsize=(15,5))
  ax = f.add_subplot(121)
  ax2 = f.add_subplot(122)

  ax.plot(H.history['loss'])
  ax.plot(H.history['val_loss'])
  ax.set_title('Model loss')
  ax.set_ylabel('Loss')
  ax.set_xlabel('Epoch')
  ax.legend(['Train', 'Validation'])
  ax.grid(True)

  # Plot training & validation accuracy values
  ax2.plot(H.history['categorical_accuracy'])
  ax2.plot(H.history['val_categorical_accuracy'])
  ax2.set_title('Model accuracy')
  ax2.set_ylabel('Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylim((0,1.1))
  ax2.legend(['Train', 'Validation'])
  ax2.grid(True)

In [ ]:
learning_curves(H)

NameError: ignored

Visualisation du résultat sur un batch d'image aléatoire

In [ ]:
import random
import numpy as np
import cv2

def depreprocess_resnet(image):
  mean = [103.939, 116.779, 123.68] # Mean pixel values of ImageNet dataset
  std = [1, 1, 1] # Standard deviation of ImageNet dataset
  unnormalized_image = np.uint8(image * std + mean)
  unnormalized_image = cv2.cvtColor(unnormalized_image, cv2.COLOR_BGR2RGB)
  return np.uint8(unnormalized_image)



def visualisation_predictions(X0,y0,y_pred):
  images = []
  images = random.sample(list(range(X0.shape[0])),9)

  labels = []
  for l in train_gen.class_indices.keys():
    labels.append(l)

  f = plt.figure(figsize=(10,10))
  for i in range(9):
    ax = f.add_subplot(431+i)

    ax.imshow(np.uint8(depreprocess_resnet(X0[images[i]])),cmap="gray")
    pred = str(labels[np.argmax(y_pred[images[i]])])
    tru  = str(labels[np.argmax(y0[images[i]])])

    #title_obj = ax.set_title("Pred=" +str(np.argmax(y_pred[images[i]]))+" True = "+str(np.argmax(y0[images[i]])))
    title_obj = ax.set_title("pred = " +pred+" / "+tru)
    plt.axis("off")
    if (np.argmax(y_pred[images[i]])!=np.argmax(y0[images[i]])):
      plt.setp(title_obj, color='r')
    else:
      plt.setp(title_obj, color='g')



In [ ]:
X0,y0 = next(val_gen)
y_pred = model.predict(X0)

visualisation_predictions(X0,y0,y_pred)

### Enregistrement du modèle

In [ ]:
model.save("myclassification_model.h5")

### Chargement d'un modèle enregistré

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("myclassification_model.h5")

## --

<font color="green">__Notebook License__</font>

Fabrice Daian (2023)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.